In [ ]:
import dataloadKuCoin as dataload
import funcsPivot as funcs
import funcsOracle as funcsOracle
import numpy as np
import pandas as pd
import random

In [ ]:
def Candle_Analysis (ticker):
    
    open_list, high_list, low_list, close_list = dataload.Load_HP(ticker)[0], dataload.Load_HP(ticker)[1], dataload.Load_HP(ticker)[2], dataload.Load_HP(ticker)[3]
    
    anomaly_eval = funcs.anomaly_eval(dataload.Load_HP(ticker), 25, 5)
    
    timeframe1 = 105
    short_list1 = close_list[(260-timeframe1):260]
    end_price, start_price, step = funcs.timeframe_slice(timeframe1)
    skeleton1 = funcs.skeleton_founder(short_list1, end_price, start_price, step)
    likelihood_dict1 = funcs.find_nature(skeleton1)[2]
    
    timeframe2 = 7
    short_list2 = close_list[(260-timeframe2):260]
    end_price, start_price, step = funcs.timeframe_slice(timeframe2)
    skeleton2 = funcs.skeleton_founder(short_list2, end_price, start_price, step)
    likelihood_dict2 = funcs.find_nature(skeleton2)[2]
    
    my_array = funcs.pl_preprocessing(open_list, high_list, low_list, close_list)
    
    isto_array = funcs.make_isto(my_array)
    
    emotion_array = funcs.final_isto(isto_array)
    
    #ALL OUTPUT
    
    #print('\n\nShort List-1:', short_list1, '\n\nShort List-2:', short_list2)
    #print('\n\nSkeleton-1:', skeleton1,'\n\nSkeleton-2:' , skeleton2)
    #print('\n\nLikelihood Dict-1:', likelihood_dict1, '\n\nLikelihood Dict-2:', likelihood_dict2)
    
    minmax_list = funcs.minmax(short_list1)
    
    df = funcs.prepare_candles(open_list, high_list, low_list, close_list)
    funcs.plot_candles(df)
    #print(funcs.support_levels(close_list))
    
    #print(emotion_array)

    funcs.plot_isto(emotion_array)
    
    k1, k2, k3, k = funcs.calculate_k(likelihood_dict1, likelihood_dict2, emotion_array)
    
    #print('K1 =', k1)
    #print('K2 =', k2)
    #print('K3 =', k3)
    #print('K =', k)
    
    #print('Anomaly k:', anomaly_eval)
    
    final_list = [k1, k2, k3, df]
    
    return final_list

In [ ]:
CH_result = Candle_Analysis ('ETH-USDT')

In [ ]:
#ORACLE FUNC

def Oracle_It(CH_result):
   
    volatility_list = funcsOracle.calculate_volatility(CH_result[3])

    #MAIN CHART
    
    united_main = funcsOracle.define_main_chart(CH_result, volatility_list)
    
    #ADDING FAIR PRICE
    
    current_price = CH_result[3]['close'][259]
    print('current_price:', current_price)
    fair_price = current_price
    
    united_main = funcsOracle.add_fair_price(CH_result, united_main, fair_price)

    #INTRADAY VOLATILITY

    intraday_volatility_list = funcsOracle.intraday_volatility(volatility_list)

    #MONTHLY VOLATILITY

    final_monthly_volatility = funcsOracle.monthly_volatility(volatility_list)
    funcs.plot_it(final_monthly_volatility)

    #RANDOMIZE

    randomized_price_list = funcsOracle.randomize_price_list(united_main, intraday_volatility_list, final_monthly_volatility)
    funcs.plot_it(randomized_price_list)

    #MAKE OHCL

    ohcl_list = funcsOracle.make_ohcl(randomized_price_list, volatility_list)
    final_ohcl_list = funcsOracle.add_price(CH_result, ohcl_list)

    #MAKE DF

    df_new = funcsOracle.make_df(final_ohcl_list)
    funcs.plot_candles(df_new)

    df_final = funcsOracle.concat_charts(CH_result, df_new)
    funcs.plot_candles(df_final)
    
    return df_final
    
price_df1 = Oracle_It(CH_result)

In [ ]:
def chart_visualization(df_final):
    
    new_array = df_final.iloc[210:310]
    new_array1 = new_array.reset_index(drop=True)
    
    new_date = [x for x in range(100)]
    new_array1['date']=new_date  
            
    funcs.plot_oracle(new_array1, '49.5', 'Ethereum', 0)
    
chart_visualization(price_df1)